# Libraries

In [1]:
import os

# go up one directory
os.chdir("..")

import pandas as pd 
import numpy as np
from tqdm import tqdm
from sklearn_extra.cluster import KMedoids
from functions import cosmic_val
from models.nmf import nmf
from functions import cosmic_val
from functions import data_handling as dh

# set seed
# np.random.seed(15)

# Data

In [2]:
data_path = "data/catalogues_Ovary_SBS.tsv"
cosmic_path = "data/COSMIC_v3.4_SBS_GRCh37.txt"
output_folder = "data/processed"
output_filename = "Ordered_Ovary_SBS.csv"
ordered_data_path = os.path.join(output_folder, output_filename)

In [3]:
dh.load_preprocess_data(data_path, cosmic_path, sep1 = "\t", sep2 = "\t", output_folder = output_folder, output_filename = output_filename)

Data already exists in  data/processed/Ordered_Ovary_SBS.csv


In [4]:
# load data
data = pd.read_csv(ordered_data_path, index_col = 0)
cosmic = pd.read_csv(cosmic_path, sep = "\t", index_col = 0)

In [5]:
LATENT_DIM = 4
TOLERANCE = 1e-4
MAX_ITERATIONS = 100_000_000

In [6]:
losses_train = []
signatures = []
iterations = 1


for i in tqdm(range(iterations)):
    
    # Applying NMF
    signatures_nmf, exposures_nmf, loss_nmf = nmf(catalog_matrix = data.to_numpy(),
                                                  num_sign = LATENT_DIM,
                                                  tol = TOLERANCE,
                                                  max_iter = MAX_ITERATIONS)
    
    # Calculating signatures and exposures for NMF
    diagonals_nmf = signatures_nmf.sum(axis=0)
    exposures_nmf = exposures_nmf.T @ np.diag(diagonals_nmf)
    signatures_nmf = (signatures_nmf) @ np.diag(1 / diagonals_nmf)
    
    losses_train.append(loss_nmf[-1])
    signatures.append(signatures_nmf)



  0%|          | 0/1 [00:00<?, ?it/s]

Iteration: 1000, Loss: 25523.80839340087
Iteration: 2000, Loss: 25233.213169672203
Iteration: 3000, Loss: 25122.936496138445
Iteration: 4000, Loss: 24933.863403054613
Iteration: 5000, Loss: 24679.16749557958
Iteration: 6000, Loss: 24345.118698119473
Iteration: 7000, Loss: 24041.229797162225
Iteration: 8000, Loss: 23914.475780511115
Iteration: 9000, Loss: 23707.28445425228
Iteration: 10000, Loss: 23130.077992118193
Iteration: 11000, Loss: 21168.461676022103
Iteration: 12000, Loss: 19555.96152841245
Iteration: 13000, Loss: 18439.641946574196
Iteration: 14000, Loss: 18400.583728475314
Iteration: 15000, Loss: 18384.547311751197
Iteration: 16000, Loss: 18372.547627583794
Iteration: 17000, Loss: 18365.830249501054
Iteration: 18000, Loss: 18359.96704635255


100%|██████████| 1/1 [00:44<00:00, 44.69s/it]


In [7]:
print("Losses train: ", np.mean(losses_train))

Losses train:  18359.17409443879


In [8]:
all_signatures = np.hstack(signatures)

In [9]:
pam = KMedoids(n_clusters = LATENT_DIM, metric='cosine').fit(all_signatures.T)
consensus_signatures = all_signatures[:, pam.medoid_indices_]

In [10]:
matched_signatures, mean_similarity = cosmic_val.compute_match(consensus_signatures, cosmic)

In [11]:
print(matched_signatures.head())
print("\nMean similarity of the matched signatures: ", mean_similarity)

   Extracted    True  Similarity
0          0  SBS10c    0.653658
1          1   SBS44    0.839025
2          2  SBS10a    0.927004
3          3  SBS40a    0.728878

Mean similarity of the matched signatures:  0.7871412027813725
